# スクレイピング

https://kamigame.jp/umamusume/page/110667391372886023.html から適正情報を取得する

## 環境構築

In [25]:
from bs4 import BeautifulSoup
import os
import pandas as pd
import re
import requests

In [26]:
DIR_OUT = '../data/scatter_scrape'
FN_UM = 'umamusume.csv'
URL = 'https://kamigame.jp/umamusume/page/110667391372886023.html'

In [27]:
# 適正一覧
INDICES= [
    'turf_index',
    'dart_index',
    'short_index',
    'mile_index',
    'intermediate_index',
    'long_index',
    'nige_index',
    'senko_index',
    'sashi_index',
    'oikomi_index',
]

## 育成ウマ娘のデータ取得

In [28]:
html = requests.get(URL)
soup = BeautifulSoup(html.text, "html.parser")

In [29]:
# ウマ娘の種別と，HTML内の位置関係
um_groups = [
    {'group': 'three-star', 'table_no': 1},
    {'group': 'two-star', 'table_no': 3},
    {'group': 'one-star', 'table_no': 5},
    {'group': 'unimplemented', 'table_no': 13},
]

In [30]:
def get_um_name(td):
    """tdよりウマ娘の名前を取得
    td = soup.select('#mainArticle > article > table')[1].\
        find_all('td')[2n]
    , where n >= 0
    """
    um_name = td.get_text().replace('\n', '')
    return um_name

In [31]:
def get_horse_name(td):
    """tdより競走馬名を取得
    td = soup.select('#mainArticle > article > table')[1].\
        find_all('td')[2n]
    , where n >= 0
    """
    um_name = get_um_name(td)
    # 例：エアグルーヴ（花嫁）からエアグルーヴのみを抽出
    horse_name = re.sub('（.*）', '', um_name)
    return horse_name

In [32]:
def get_um_indices(td):
    """tdよりウマ娘の適正値を取得
        td = soup.select('#mainArticle > article > table')[1].\
        find_all('td')[2n+1]
    , where n >= 0
    """
    imgs = td.find_all('img')
    alts = [i.get('alt') for i in imgs]
    indices = {
        i: v for i, v in zip(INDICES, alts)
    }
    return indices

In [33]:
# ウマ娘の情報を格納
ums = []

for g in um_groups:
    group = g['group']
    i_table = g['table_no']
    
    tds = soup.select('#mainArticle > article > table')[i_table].\
        find_all('td')
    assert len(tds) % 2 == 0
    n_um = len(tds) // 2
    
    if group != 'umimplemented':
        # 実装されたウマ娘は，偶数番目から名称，奇数番目から適正を抽出する
        for n in range(len(tds) // 2):
            um = {
                'um_name': get_um_name(tds[2 * n]),
                'horse_name': get_horse_name(tds[2 * n])}
            um.update(get_um_indices(tds[2 * n + 1]))
            um['group'] = group
            ums.append(um)
    else:
        # 未実装のウマ娘には適正情報がない
        for n in range(len(tds)):
            um = {'um_name': get_um_name(tds[n]),
                  'horse_name': get_horse_name(tds[n])}
            um['group'] = group
            ums.append(um)

In [34]:
# DataFrame化
df_um = pd.DataFrame(ums)

In [35]:
# 新衣装等の追加でウマ娘の適正が変わっていないか，念の為確認
# 例：エアグルーヴ vs エアグルーヴ（花嫁）

## horse_nameが重複している数を集計
df_counts = df_um['horse_name'].value_counts().reset_index()
df_counts.columns = ['horse_name', 'counts']

for r in df_counts.to_dict('records'):
    if r['counts'] > 1:
        df_tmp = df_um[df_um['horse_name']==r['horse_name']].\
            reset_index(drop=True)
        # 重複した行数
        n_dup = df_tmp[INDICES].duplicated().sum()
        # 最初の1行以外全て重複している（脚質等の情報が変わっていない）か
        assert df_tmp.shape[0] - n_dup == 1

大丈夫そう

In [36]:
# こんな感じのDataFrameになる
df_um.head().T

,0,1,2,3,4
um_name,アグネスデジタル,エアグルーヴ（花嫁）,エイシンフラッシュ,エルコンドルパサー（新衣装）,オグリキャップ
horse_name,アグネスデジタル,エアグルーヴ,エイシンフラッシュ,エルコンドルパサー,オグリキャップ
turf_index,A,A,A,A,A
dart_index,A,G,G,B,B
short_index,F,C,G,F,E
mile_index,A,B,F,A,A
intermediate_index,A,A,A,A,A
long_index,G,E,A,B,B
nige_index,G,D,G,E,F
senko_index,A,A,B,A,A


In [37]:
# 保存
df_um.to_csv(os.path.join(DIR_OUT, 'umamusume.csv'), index=False)